![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [152]:
import pandas as pd
import numpy as np

In [153]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [154]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


Let's form the client table first and clean it:

In [155]:
client = df.loc[:, 'client_id':'mortgage']
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [156]:
client['job'] = client['job'].str.replace('\.', '_')
client['job'].head(10)

0      housemaid
1       services
2       services
3         admin_
4       services
5       services
6         admin_
7    blue-collar
8     technician
9       services
Name: job, dtype: object

In [157]:
client['education'].head(15)

0                basic.4y
1             high.school
2             high.school
3                basic.6y
4             high.school
5                basic.9y
6     professional.course
7                 unknown
8     professional.course
9             high.school
10                unknown
11            high.school
12            high.school
13               basic.4y
14               basic.6y
Name: education, dtype: object

In [158]:
client['education'] = client['education'].str.replace('\.', '_')
nulls = ['unknown']
client.loc[client['education'].isin(nulls), 'education'] = np.nan
client['education'].head(15)

0                basic_4y
1             high_school
2             high_school
3                basic_6y
4             high_school
5                basic_9y
6     professional_course
7                     NaN
8     professional_course
9             high_school
10                    NaN
11            high_school
12            high_school
13               basic_4y
14               basic_6y
Name: education, dtype: object

In [159]:
client['credit_default'].value_counts()

no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64

In [160]:
pos = ['yes']
false = ['unknown', 'no']
client.loc[client['credit_default'].isin(pos), 'credit_default'] = 1
client.loc[client['credit_default'].isin(false), 'credit_default'] = 0 
client['credit_default'] = client['credit_default'].astype(bool)
client['credit_default'].value_counts()

False    41185
True         3
Name: credit_default, dtype: int64

In [161]:
client['mortgage'].value_counts()

yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64

In [162]:
pos = ['yes']
false = ['unknown', 'no']
client.loc[client['mortgage'].isin(pos), 'mortgage'] = 1
client.loc[client['mortgage'].isin(false), 'mortgage'] = 0 
client['mortgage'] = client['mortgage'].astype(bool)
client['mortgage'].value_counts()

True     21576
False    19612
Name: mortgage, dtype: int64

In [163]:
client.head(10)

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
5,5,45,services,married,basic_9y,False,False
6,6,59,admin_,married,professional_course,False,False
7,7,41,blue-collar,married,NaN,False,False
8,8,24,technician,single,professional_course,False,True
9,9,25,services,single,high_school,False,True


Moving on to the campaign table

In [164]:
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


Let's make the last_contact_date column

In [165]:
df['month'].value_counts()

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64

In [166]:
month_to_number = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}

df['month'] = df['month'].map(month_to_number)
df['year'] = '2022'

df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year
0,0,56,housemaid,married,basic.4y,no,no,5,13,261,1,0,nonexistent,93.994,4.857,no,2022
1,1,57,services,married,high.school,unknown,no,5,19,149,1,0,nonexistent,93.994,4.857,no,2022
2,2,37,services,married,high.school,no,yes,5,23,226,1,0,nonexistent,93.994,4.857,no,2022
3,3,40,admin.,married,basic.6y,no,no,5,27,151,1,0,nonexistent,93.994,4.857,no,2022
4,4,56,services,married,high.school,no,no,5,3,307,1,0,nonexistent,93.994,4.857,no,2022


In [167]:
campaign['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']], format='%Y %B %d')
#submission only accepted the datetime format with time tickets, code below would turn it tp 'YYYY-MM-DD'
#campaign['last_contact_date'] = campaign['last_contact_date'].dt.strftime('%Y-%m-%d')
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,nonexistent,no,2022-05-13
1,1,1,149,0,nonexistent,no,2022-05-19
2,2,1,226,0,nonexistent,no,2022-05-23
3,3,1,151,0,nonexistent,no,2022-05-27
4,4,1,307,0,nonexistent,no,2022-05-03


In [169]:
campaign['previous_outcome'].value_counts()

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

In [170]:
succ = ['success']
fail = ['failure', 'nonexistent']

campaign.loc[campaign['previous_outcome'].isin(succ), 'previous_outcome'] = 1
campaign.loc[campaign['previous_outcome'].isin(fail), 'previous_outcome'] = 0 

campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)
campaign['previous_outcome'].value_counts()

False    39815
True      1373
Name: previous_outcome, dtype: int64

In [171]:
campaign['campaign_outcome'].value_counts()

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [172]:
yes = ['yes']
no = ['no']

campaign.loc[campaign['campaign_outcome'].isin(yes), 'campaign_outcome'] = 1
campaign.loc[campaign['campaign_outcome'].isin(no), 'campaign_outcome'] = 0 

campaign['campaign_outcome'] = campaign['campaign_outcome'].astype(bool)
campaign['campaign_outcome'].value_counts()

False    36548
True      4640
Name: campaign_outcome, dtype: int64

Last one, economics

In [173]:
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [174]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)